# Ligand binding affinity prediction using the Random Forest algorithm
#### Laksh Aithani, University of Cambridge, 2019

### Imports

In [1]:
# Imports
import os

# RDkit, a chemoinformatics library
import rdkit
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import PandasTools

import numpy as np

import pandas as pd

# Machine learning imports
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

### Loading in dataset; needs to be in current directory
### The dataset was obtained from https://www.bindingdb.org/bind/index.jsp

In [2]:
# Load in file; for this to work, csv file needs to be in your current directory/folder you're working in.
file_name = os.path.join(os.getcwd(),'serotonin1a.csv')
df = pd.read_csv(file_name)

#Inspect dataframe. Note the SMILES column and the Ki column
df.head(2)

,BindingDB Reactant_set_id,Ligand SMILES,Ligand InChI,Ligand InChI Key,BindingDB MonomerID,BindingDB Ligand Name,Target Name Assigned by Curator or DataSource,Target Source Organism According to Curator or DataSource,Ki (nM),IC50 (nM),...,UniProt (SwissProt) Recommended Name of Target Chain,UniProt (SwissProt) Entry Name of Target Chain,UniProt (SwissProt) Primary ID of Target Chain,UniProt (SwissProt) Secondary ID(s) of Target Chain,UniProt (SwissProt) Alternative ID(s) of Target Chain,UniProt (TrEMBL) Submitted Name of Target Chain,UniProt (TrEMBL) Entry Name of Target Chain,UniProt (TrEMBL) Primary ID of Target Chain,UniProt (TrEMBL) Secondary ID(s) of Target Chain,UniProt (TrEMBL) Alternative ID(s) of Target Chain
0,50645342,O=c1n(CCN2CCN(CC2)c2cccc3ccccc23)nnc2ccccc12,InChI=1S/C23H23N5O/c29-23-20-9-3-4-10-21(20)24...,JQQXUGWNTUCGOI-UHFFFAOYSA-N,50346374,3-(2-(4-(naphthalen-1-yl)piperazin-1-yl)ethyl)...,Serotonin (5-HT) receptor,Rattus norvegicus,0.000178,NaN,...,5-hydroxytryptamine receptor 1A,5HT1A_RAT,P19327,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,50060249,O=C1[C@H]2[C@@H]3CC[C@@H](C3)[C@H]2C(=O)N1CCCC...,InChI=1S/C21H29N5O2/c27-19-17-15-4-5-16(14-15)...,CEIJFEGBUDEYSX-FZDBZEDMSA-N,50368723,Metanopirone::Sediel::TANDOSPIRONE HYDROCHLORI...,Serotonin (5-HT) receptor,Rattus norvegicus,0.0027,NaN,...,5-hydroxytryptamine receptor 1A,5HT1A_RAT,P19327,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# Converting strings to floats, also set all non-numbers to NaN
df['Ki (nM)'] = pd.to_numeric(df['Ki (nM)'],errors = 'coerce')

# Drop Nans
df.dropna(subset = ['Ki (nM)'], inplace = True)
df.reset_index(inplace = True)
df = df.drop('index',axis = 1)

# Inspect dataframe
df.head(2)

,BindingDB Reactant_set_id,Ligand SMILES,Ligand InChI,Ligand InChI Key,BindingDB MonomerID,BindingDB Ligand Name,Target Name Assigned by Curator or DataSource,Target Source Organism According to Curator or DataSource,Ki (nM),IC50 (nM),...,UniProt (SwissProt) Recommended Name of Target Chain,UniProt (SwissProt) Entry Name of Target Chain,UniProt (SwissProt) Primary ID of Target Chain,UniProt (SwissProt) Secondary ID(s) of Target Chain,UniProt (SwissProt) Alternative ID(s) of Target Chain,UniProt (TrEMBL) Submitted Name of Target Chain,UniProt (TrEMBL) Entry Name of Target Chain,UniProt (TrEMBL) Primary ID of Target Chain,UniProt (TrEMBL) Secondary ID(s) of Target Chain,UniProt (TrEMBL) Alternative ID(s) of Target Chain
0,50645342,O=c1n(CCN2CCN(CC2)c2cccc3ccccc23)nnc2ccccc12,InChI=1S/C23H23N5O/c29-23-20-9-3-4-10-21(20)24...,JQQXUGWNTUCGOI-UHFFFAOYSA-N,50346374,3-(2-(4-(naphthalen-1-yl)piperazin-1-yl)ethyl)...,Serotonin (5-HT) receptor,Rattus norvegicus,0.000178,NaN,...,5-hydroxytryptamine receptor 1A,5HT1A_RAT,P19327,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,50060249,O=C1[C@H]2[C@@H]3CC[C@@H](C3)[C@H]2C(=O)N1CCCC...,InChI=1S/C21H29N5O2/c27-19-17-15-4-5-16(14-15)...,CEIJFEGBUDEYSX-FZDBZEDMSA-N,50368723,Metanopirone::Sediel::TANDOSPIRONE HYDROCHLORI...,Serotonin (5-HT) receptor,Rattus norvegicus,0.002700,NaN,...,5-hydroxytryptamine receptor 1A,5HT1A_RAT,P19327,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Grab the relevant columns, that is, our training variable (SMILES string) and our variable we are trying to predict (Ki (nM)

In [4]:
# Get SMILES for conversion into Morgan string
smiles = df['Ligand SMILES']

# Get binding constant to receptor
Ki = df['Ki (nM)']

print(f"Max Ki:{Ki.max():.3g} nM, Min Ki:{Ki.min():.3g} nM, Number of compounds: {len(Ki)}")

Max Ki:4.37e+08 nM, Min Ki:0.000178 nM, Number of compounds: 7292


## We now have to get our data into the familiar machine learning format, that is, a matrix.
### We'll do this using RDkit's built-in Morgan fingerprint algorithm to convert a SMILES string into a 2048-length vector, based on the structure of the molecule. For more details, see here:
#### J. Chem. Inf. Model.  50, 5, 742-754
https://pubs.acs.org/doi/abs/10.1021/ci100050t


In [5]:
# Generate the Morgan matrix; takes about 1 minute

Morgan_matrix = np.zeros((1,2048))
l = len(smiles)

#Iterate through the compounds
for i in range(l):
    
    # For each compound, get the structure, convert to Morgan fingerprint,
    # and add to data matrix.
    
    compound = Chem.MolFromSmiles(smiles[i])
    fp = Chem.AllChem.GetMorganFingerprintAsBitVect(compound, 2, nBits = 2048)
    fp = fp.ToBitString()
    matrix_row = np.array([int(x) for x in list(fp)])
    Morgan_matrix = np.row_stack((Morgan_matrix, matrix_row))
    
    # Progress checker
    if i%500==0:
        percentage = np.round(100* (i/l),1)
        print(f"{percentage}% done")

Morgan_matrix = np.delete(Morgan_matrix,0,axis = 0)

print('\n')
print(f"Matrix dimension:{Morgan_matrix.shape}")

0.0% done
6.9% done
13.7% done
20.6% done
27.4% done
34.3% done
41.1% done
48.0% done
54.9% done
61.7% done
68.6% done
75.4% done
82.3% done
89.1% done
96.0% done


Matrix dimension:(7292, 2048)


# We'll now train a Random Forest algorithm on this data
### We'll use 10 estimators

In [6]:
# Initiate the model
reg = RandomForestRegressor(n_estimators = 10)

# Split the data into a training and testing set to check for overfitting
X_train, X_test, y_train, y_test = train_test_split(Morgan_matrix,Ki, random_state = 3)

#Train the model; takes around 1 minute
reg.fit(X_train, y_train)

# Predict testing set using trained model
y_pred = reg.predict(X_test)

## View how our model performed

In [7]:
# Make a new dataframe to view the results
df2 = y_test.reset_index()
df2['pred'] = y_pred

# Pretty good predictions as one can see!
df2.head(50)

,index,Ki (nM),pred
0,2152,2000.000,301.350333
1,6743,69.700,845.365250
2,3455,0.200,30.801000
3,1201,397.900,1403.950000
4,467,180.000,517.623375
5,4830,13.000,31.675833
6,1173,1.000,14.338000
7,529,192.000,179.240000
8,7017,90.000,367.612093
9,7117,99.000,282.220000


## Let's see how our model classified the drugs
### Call a drug a strong binder if its affinity is less than 5nM

In [8]:
# Test classification accuracy by defining an affinity of 
# 5nM or below as being a strong binder, and above
# 5nM as being a weak binder

tests = df2['Ki (nM)'] 
preds = df2['pred']

tests = tests>5
preds = preds>5
cm = confusion_matrix(tests,preds)
print(cm)
print('\n')
print(classification_report(tests,preds))
print('A very significant improvement over a 50% guess rate')

[[ 125  283]
 [  34 1381]]


              precision    recall  f1-score   support

       False       0.79      0.31      0.44       408
        True       0.83      0.98      0.90      1415

   micro avg       0.83      0.83      0.83      1823
   macro avg       0.81      0.64      0.67      1823
weighted avg       0.82      0.83      0.79      1823

A very significant improvement over a 50% guess rate
